#### SIPP1991 401K data

#### logs

5/15/2024 WL: template provided. TODO: write a brief intro to the data discussing the background, and the meaning of the variables; do a bit data summary and regression along the similar lines of ldw data.


Data source: SIPP 1991 (Abadie, 2003), pages 231-263  
This paper investigates the effect of 401(k) programs on savings. To combat selection bias, they use a new statistical method that accounts for individual differences in benefit from 401(k), works with continuous outcomes (like amount saved) and avoids strong assumptions on variable relationships.   
The variables in the data set include:
\begin{align*}
%\begin{array}{ll}
\hline \text { Variable } & \quad \text { Description } \\
\hline \text { net tfa } &\quad  \text { Net total financial assets } \\
\text { e401 } &\quad  \text { = 1 if employers offers 401(k) } \\
\text { p401 } &\quad  \text { = 1 if the employee participates 401(k) } \\
\text { age } & \quad \text { age } \\
\text { inc } &\quad  \text { income } \\
\text { fsize } &\quad  \text { family size } \\
\text { educ } &\quad  \text { years of education }\\
\text { db } &\quad  \text { = 1 if indivudual has defined benefit pension } \\
\text { marr } &\quad \text { = 1 if married } \\
\text { twoearn } &\quad \text { = 1 if two-earner household } \\
\text { pira } &\quad \text { = 1 if individual participates in IRA } \\
\text { hown } &\quad \text { = 1 if home owner } \\
\hline
%\end{array}
\end{align*}

The data consist of 9275 observations from the Survey of Income and Program Participation (SIPP) of 1991. It is restricted to households of persons aged 25-64, where at least one individual is employed and no persns are self-employed. Additionally, family income ranges beteen $10,000 - $200,000.

In [2]:
import io
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests
from data_process import load_rdata

# Print the current working directory
print("Current working directory: {0}".format(os.getcwd()))

Current working directory: c:\Users\philh\Projects\One\401K


In [19]:
fin_data = load_rdata(r"C:\Users\philh\Projects\One\401K\data\sipp1991.Rdata")
df = list(fin_data.values())[0].drop(["nifa", "tw","age","fsize","educ","db","marr","twoearn","hown"], axis=1)
df.head()

,net_tfa,inc,e401,p401,pira
rownames,,,,,
1,0.0,6765.0,0,0,0
2,1015.0,28452.0,0,0,0
3,-2000.0,3300.0,0,0,0
4,15000.0,52590.0,0,0,0
5,0.0,21804.0,0,0,0


In [22]:
#Statistics
print(df.describe())
print(sum(df["net_tfa"]==0), sum(df["p401"]==0))


            net_tfa            inc         e401         p401         pira
count  9.915000e+03    9915.000000  9915.000000  9915.000000  9915.000000
mean   1.805153e+04   37200.623197     0.371357     0.261624     0.242158
std    6.352250e+04   24774.288006     0.483192     0.439541     0.428411
min   -5.023020e+05   -2652.000000     0.000000     0.000000     0.000000
25%   -5.000000e+02   19413.000000     0.000000     0.000000     0.000000
50%    1.499000e+03   31476.000000     0.000000     0.000000     0.000000
75%    1.652450e+04   48583.500000     1.000000     1.000000     0.000000
max    1.536798e+06  242124.000000     1.000000     1.000000     1.000000
783 7321
